In [ ]:
# create a character level Fuel dataset from text file

%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [1]:
import os
import codecs
import subprocess
from subprocess import Popen, PIPE, STDOUT

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

import numpy as np
from abc import ABCMeta, abstractmethod
from toolz import merge
import pandas as pd
from six import add_metaclass
from picklable_itertools.extras import equizip
from blocks.bricks.recurrent import (GatedRecurrent, Bidirectional)
from blocks.initialization import IsotropicGaussian, Constant, Orthogonal
import theano
import theano.tensor as T

from blocks.bricks import Initializable
from blocks.bricks.base import application, Brick, lazy
from blocks.bricks import Tanh, Linear, MLP
from blocks.bricks.lookup import LookupTable
from blocks.bricks.parallel import Fork
from blocks.utils import (shared_floatx_nans, dict_union)
from blocks.roles import add_role, WEIGHT


from fuel.datasets import TextFile
from fuel.schemes import ConstantScheme
from fuel.streams import DataStream
from fuel.transformers import (
    Merge, Batch, Filter, Padding, SortMapping, Unpack, Mapping)

DEBUG:root:test


In [6]:
# path to your training data
TRAINING_DATASET = '/home/chris/projects/machine_learning/dcu-character-lms/data/paul_graham_essays.txt'

UNKNOWN_TOKEN = '_UNK_'
EOS_TOKEN = '</S>'

In [7]:
all_symbols = set()

# make our dictionaries, and write a temporary file for the 
with codecs.open(TRAINING_DATASET, encoding='utf8') as inp:
    for l in inp.read().strip().split('\n'):
        all_symbols.update(l)
        y_seq = l[1:] + [EOS_TOKEN]
        assert len(l) == len(y_seq)
        
# add an unknown token in case we observe a new character at prediction time
all_symbols.update([UNKNOWN_TOKEN])
        
len(all_symbols)

51

In [5]:
word2idx = {v:k for k,v in enumerate(all_symbols)}
idx2word = {v:k for k,v in word2idx.items()}


In [ ]:
train_X = TextFile([TRAINING_DATASET], word2idx, bos_token=None, eos_token=None,
                   unk_token=UNKNOWN_TOKEN)

train_Y = TextFile([x2_file], vocab, bos_token=None, eos_token=None, unk_token=unk_token)

# Merge them to get x1, x2 pairs
# stream = Merge([x1_dataset.get_example_stream(),
#                 x2_dataset.get_example_stream(),
#                 score_dataset.get_example_stream()],
#                 ('x1', 'x2', 'y'))